#**LLM**

In [38]:
!pip install groq colorama

In [39]:
from google.colab import userdata
groq_api_key = userdata.get('groq_api_key')

In [40]:
import os
from pprint import pprint
from groq import Groq
from IPython.display import display_markdown
import os
import re
import json
from typing import Callable
import time

from colorama import Fore
from colorama import Style

In [19]:
# MODEL = "llama3-groq-70b-8192-tool-use-preview"
# MODEL = "deepseek-r1-distill-llama-70b"
MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT =  Groq(api_key=groq_api_key)

#**Function --> tool**

In [12]:

def get_current_weather(location: str, unit: str):
	"""
	Get the current weather in a given location

	location (str): The city and state, e.g. Madrid, Barcelona
	unit (str): The unit. It can take two values; "celsius", "fahrenheit"
	"""
	if location == "Madrid":
		return json.dumps({"temperature": 25, "unit": unit})

	else:
		return json.dumps({"temperature": 58, "unit": unit})

In [13]:
get_current_weather(location="Madrid", unit="celsius")


'{"temperature": 25, "unit": "celsius"}'

#**tool call agent**

##**system prompt**
manually pass the tool signature to the prompt

In [20]:
# Define the System Prompt as a constant

TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>
"""

##**Chat History**
1. Tool chat history
2. agent chat history

In [21]:
tool_chat_history = [
    {
        "role": "system",
        "content": TOOL_SYSTEM_PROMPT
    }
]


agent_chat_history = []

# User Query
user_msg = {
    "role": "user",
    "content": "What's the current temperature in Madrid, in Celsius?"
}


tool_chat_history.append(user_msg)
agent_chat_history.append(user_msg)


In [22]:
output = GROQ_CLIENT.chat.completions.create(
    messages=tool_chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<tool_call>
{"name": "get_current_weather", "arguments": {"location": "Madrid", "unit": "celsius"}}
</tool_call>


##**Tool parser**

In [23]:
def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)

    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        print(f"Unexpected error: {e}")
        return "There was some error parsing the Tool's output"

In [24]:
parsed_output = parse_tool_call_str(output)
parsed_output

{'name': 'get_current_weather',
 'arguments': {'location': 'Madrid', 'unit': 'celsius'}}

##**Observation**
observation is the output of tool runner

In [26]:
result = get_current_weather(**parsed_output["arguments"])
result

'{"temperature": 25, "unit": "celsius"}'

##**Appending observation to agent history**

In [27]:
agent_chat_history.append({
    "role": "user",
    "content": f"Observation: {result}"
})

##**Final step --> llm call with the agent history**

In [28]:
GROQ_CLIENT.chat.completions.create(
    messages=agent_chat_history,
    model=MODEL
).choices[0].message.content

'The current temperature in Madrid is 25°C.'

#**All together**

##**Completion from llm**

In [29]:
def completions_create(client, messages: list, model: str) -> str:
    """
    Sends a request to the client's `completions.create` method to interact with the language model.

    Args:
        client (Groq): The Groq client object
        messages (list[dict]): A list of message objects containing chat history for the model.
        model (str): The model to use for generating tool calls and responses.

    Returns:
        str: The content of the model's response.
    """
    response = client.chat.completions.create(messages=messages, model=model)
    return str(response.choices[0].message.content)

##**prompt**

In [41]:
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call>
XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>,  "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools:

<tools>
%s
</tools>
"""

In [42]:
def build_prompt_structure(prompt: str, role: str, tag: str = "") -> dict:
    """
    Builds a structured prompt that includes the role and content.

    Args:
        prompt (str): The actual content of the prompt.
        role (str): The role of the speaker (e.g., user, assistant).

    Returns:
        dict: A dictionary representing the structured prompt.
    """
    if tag:
        prompt = f"<{tag}>{prompt}</{tag}>"
    return {"role": role, "content": prompt}

##**Chat history**

In [43]:
class ChatHistory(list):
    def __init__(self, messages: list | None = None, total_length: int = -1):
        """Initialise the queue with a fixed total length.

        Args:
            messages (list | None): A list of initial messages
            total_length (int): The maximum number of messages the chat history can hold.
        """
        if messages is None:
            messages = []

        super().__init__(messages)
        self.total_length = total_length

    def append(self, msg: str):
        """Add a message to the queue.

        Args:
            msg (str): The message to be added to the queue
        """
        if len(self) == self.total_length:
            self.pop(0)
        super().append(msg)


In [44]:
def update_chat_history(history: list, msg: str, role: str):
    """
    Updates the chat history by appending the latest response.

    Args:
        history (list): The list representing the current chat history.
        msg (str): The message to append.
        role (str): The role type (e.g. 'user', 'assistant', 'system')
    """
    history.append(build_prompt_structure(prompt=msg, role=role))


##**Tool utils**

###**Function json output**

<tools> {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location location (str): The city and state, e.g. Madrid, Barcelona unit (str): The unit. It can take two values; 'celsius', 'fahrenheit'",
    "parameters": {
        "properties": {
            "location": {
                "type": "str"
            },
            "unit": {
                "type": "str"
            }
        }
    }
}
</tools>

In [32]:
def get_fn_signature(fn: Callable) -> dict:
    """
    Generates the signature for a given function.

    Args:
        fn (Callable): The function whose signature needs to be extracted.

    Returns:
        dict: A dictionary containing the function's name, description,
              and parameter types.
    """
    fn_signature: dict = {
        "name": fn.__name__,
        "description": fn.__doc__,
        "parameters": {"properties": {}},
    }
    schema = {
        k: {"type": v.__name__} for k, v in fn.__annotations__.items() if k != "return"
    }
    fn_signature["parameters"]["properties"] = schema
    return fn_signature


In [33]:
get_fn_signature(get_current_weather)

{'name': 'get_current_weather',
 'description': '\n\tGet the current weather in a given location\n\n\tlocation (str): The city and state, e.g. Madrid, Barcelona\n\tunit (str): The unit. It can take two values; "celsius", "fahrenheit"\n\t',
 'parameters': {'properties': {'location': {'type': 'str'},
   'unit': {'type': 'str'}}}}

###**tool class and decorator**

In [34]:
class Tool:
    """
    A class representing a tool that wraps a callable and its signature.

    Attributes:
        name (str): The name of the tool (function).
        fn (Callable): The function that the tool represents.
        fn_signature (str): JSON string representation of the function's signature.
    """

    def __init__(self, name: str, fn: Callable, fn_signature: str):
        self.name = name  # tool name
        self.fn = fn      # callable function
        self.fn_signature = fn_signature  # output from the previous function

    def __str__(self):
        return self.fn_signature

    def run(self, **kwargs):
        """
        Executes the tool (function) with provided arguments.

        Args:
            **kwargs: Keyword arguments passed to the function.

        Returns:
            The result of the function call.
        """
        return self.fn(**kwargs)

In [36]:
def tool(fn: Callable):
    """
    A decorator that wraps a function into a Tool object.

    Args:
        fn (Callable): The function to be wrapped.

    Returns:
        Tool: A Tool object containing the function, its name, and its signature.
    """

    def wrapper():
        fn_signature = get_fn_signature(fn)
        return Tool(
            name=fn_signature.get("name"),
            fn=fn,
            fn_signature=json.dumps(fn_signature)
        )

    return wrapper()

###**Arguments Validator**

In [46]:
def validate_arguments(tool_call: dict, tool_signature: dict) -> dict:
    """
    Validates and converts arguments in the input dictionary to match the expected types.

    Args:
        tool_call (dict): A dictionary containing the arguments passed to the tool.
        tool_signature (dict): The expected function signature and parameter types.

    Returns:
        dict: The tool call dictionary with the arguments converted to the correct types if necessary.
    """
    properties = tool_signature["parameters"]["properties"]

    # TODO: This is overly simplified but enough for simple Tools.
    type_mapping = {
        "int": int,
        "str": str,
        "bool": bool,
        "float": float,
    }

    for arg_name, arg_value in tool_call["arguments"].items():
        expected_type = properties[arg_name].get("type")

        # check if output arg value from the model != expected
        if not isinstance(arg_value, type_mapping[expected_type]):
            tool_call["arguments"][arg_name] = type_mapping[expected_type](arg_value)

    return tool_call

##**Tag exctractor**

In [48]:

import re
from dataclasses import dataclass


@dataclass
class TagContentResult:
    """
    A data class to represent the result of extracting tag content.

    Attributes:
        content (List[str]): A list of strings containing the content found between the specified tags.
        found (bool): A flag indicating whether any content was found for the given tag.
    """

    content: list[str]
    found: bool


def extract_tag_content(text: str, tag: str) -> TagContentResult:
    """
    Extracts all content enclosed by specified tags (e.g., <thought>, <response>, etc.).

    Parameters:
        text (str): The input string containing multiple potential tags.
        tag (str): The name of the tag to search for (e.g., 'thought', 'response').

    Returns:
        dict: A dictionary with the following keys:
            - 'content' (list): A list of strings containing the content found between the specified tags.
            - 'found' (bool): A flag indicating whether any content was found for the given tag.
    """
    # Build the regex pattern dynamically to find multiple occurrences of the tag
    tag_pattern = rf"<{tag}>(.*?)</{tag}>"

    # Use findall to capture all content between the specified tag
    matched_contents = re.findall(tag_pattern, text, re.DOTALL)

    # Return the dataclass instance with the result
    return TagContentResult(
        content=[content.strip() for content in matched_contents],
        found=bool(matched_contents),
    )


##**Tool Agent class**

In [67]:
class ToolAgent:
    """
    The ToolAgent class represents an agent that can interact with a language model and use tools
    to assist with user queries. It generates function calls based on user input, validates arguments,
    and runs the respective tools.

    Attributes:
        tools (Tool | list[Tool]): A list of tools available to the agent.
        model (str): The model to be used for generating tool calls and responses.
        client (Groq): The Groq client used to interact with the language model.
        tools_dict (dict): A dictionary mapping tool names to their corresponding Tool objects.
    """

    def __init__(self, tools: Tool | list[Tool], model: str = "llama-3.3-70b-versatile") -> None:
        self.client = Groq(api_key=groq_api_key)
        self.model = model
        self.tools = tools if isinstance(tools, list) else [tools] # force to be a list
        self.tools_dict = {tool.name: tool for tool in self.tools}

    def add_tool_signatures(self) -> str:
        """
        Collects the function signatures of all available tools.

        Returns:
            str: A concatenated string of all tool function signatures in JSON format.
        """
        return "".join([tool.fn_signature for tool in self.tools])

    def process_tool_calls(self, tool_calls_content: list) -> dict:
        """
        Processes each tool call, validates arguments, executes the tools, and collects results.

        Args:
            tool_calls_content (list): List of strings, each representing a tool call in JSON format.

        Returns:
            dict: A dictionary where the keys are tool call IDs and values are the results from the tools.
        """
        observations = {}
        # tool call content is in json format returning from the model
        # we want : tool name , tool
        # we have tool dict {"tool_name" : tool }
        for tool_call_str in tool_calls_content:
            tool_call = json.loads(tool_call_str)
            tool_name = tool_call["name"]
            tool = self.tools_dict[tool_name]

            print(Fore.GREEN + f"\nUsing Tool: {tool_name}")

            # Validate and execute the tool call
            validated_tool_call = validate_arguments(
                tool_call, json.loads(tool.fn_signature)
            )
            print(Fore.GREEN + f"\nTool call dict: \n{validated_tool_call}")

            result = tool.run(**validated_tool_call["arguments"])
            print(Fore.GREEN + f"\nTool result: \n{result}")

            # Store the result using the tool call ID
            observations[validated_tool_call["id"]] = result

        return observations

    def run(self, user_msg: str) -> str:
        """
        Handles the full process of interacting with the language model and executing a tool based on user input.

        Args:
            user_msg (str): The user's message that prompts the tool agent to act.

        Returns:
            str: The final output after executing the tool and generating a response from the model.
        """
        # build user query prompt
        user_prompt = build_prompt_structure(prompt=user_msg, role="user")

        # append system msg and user query to tool chat history
        tool_chat_history = ChatHistory(
            [
                build_prompt_structure(
                    prompt=TOOL_SYSTEM_PROMPT % self.add_tool_signatures(),
                    role="system",
                ),
                user_prompt,
            ]
        )

        # append user query to the agent chat hsitory
        agent_chat_history = ChatHistory([user_prompt])

        # tall call output --> tool name + tool args
        tool_call_response = completions_create(
            self.client, messages=tool_chat_history, model=self.model
        )

        # tool_call_response from the llm model  <tool_call> </tool_call>
        # also pass the tag = <tool_call>
        # Extract_tag_content returns:
        # - 'content' (list): A list of strings containing the content found between the specified tags.
        # - 'found' (bool): A flag indicating whether any content was found for the given tag.
        tool_calls = extract_tag_content(str(tool_call_response), "tool_call")

        # getting observation to add to the agent history
        if tool_calls.found:
            observations = self.process_tool_calls(tool_calls.content)
            update_chat_history(
                agent_chat_history, f'f"Observation: {observations}"', "user"
            )

        # final step to generate final response from the observation
        return completions_create(self.client, agent_chat_history, self.model)

#**Testing**

##**Tool to use**

In [68]:
import requests


def fetch_top_hacker_news_stories(top_n: int):
    """
    Fetch the top stories from Hacker News.

    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API.
    Each story contains the title, URL, score, author, and time of submission. The data is fetched
    from the official Firebase Hacker News API, which returns story details in JSON format.

    Args:
        top_n (int): The number of top stories to retrieve.
    """
    top_stories_url = 'https://hacker-news.firebaseio.com/v0/topstories.json'

    try:
        response = requests.get(top_stories_url)
        response.raise_for_status()  # Check for HTTP errors

        # Get the top story IDs
        top_story_ids = response.json()[:top_n]

        top_stories = []

        # For each story ID, fetch the story details
        for story_id in top_story_ids:
            story_url = f'https://hacker-news.firebaseio.com/v0/item/{story_id}.json'
            story_response = requests.get(story_url)
            story_response.raise_for_status()  # Check for HTTP errors
            story_data = story_response.json()

            # Append the story title and URL (or other relevant info) to the list
            top_stories.append({
                'title': story_data.get('title', 'No title'),
                'url': story_data.get('url', 'No URL available'),
            })

        return json.dumps(top_stories)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

In [70]:
hn_tool = tool(fetch_top_hacker_news_stories)
hn_tool

In [71]:
hn_tool.name


'fetch_top_hacker_news_stories'

In [72]:
hn_tool.fn_signature

'{"name": "fetch_top_hacker_news_stories", "description": "\\n    Fetch the top stories from Hacker News.\\n\\n    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. \\n    Each story contains the title, URL, score, author, and time of submission. The data is fetched \\n    from the official Firebase Hacker News API, which returns story details in JSON format.\\n\\n    Args:\\n        top_n (int): The number of top stories to retrieve.\\n    ", "parameters": {"properties": {"top_n": {"type": "int"}}}}'

In [73]:
json.loads(hn_tool.fn_signature)


{'name': 'fetch_top_hacker_news_stories',
 'description': '\n    Fetch the top stories from Hacker News.\n\n    This function retrieves the top `top_n` stories from Hacker News using the Hacker News API. \n    Each story contains the title, URL, score, author, and time of submission. The data is fetched \n    from the official Firebase Hacker News API, which returns story details in JSON format.\n\n    Args:\n        top_n (int): The number of top stories to retrieve.\n    ',
 'parameters': {'properties': {'top_n': {'type': 'int'}}}}

##**Agent instant**

In [74]:
tool_agent = ToolAgent(tools=[hn_tool])


In [80]:
output = tool_agent.run(user_msg="Tell me the top 5 Hacker News stories right now")



Using Tool: fetch_top_hacker_news_stories

Tool call dict: 
{'name': 'fetch_top_hacker_news_stories', 'arguments': {'top_n': 5}, 'id': 1}

Tool result: 
[{"title": "Taking a $15 Casio F91W 5km underwater", "url": "https://www.watchesofespionage.com/blogs/woe-dispatch/casio-f91w-diving-underwater-pressure-test"}, {"title": "Images Reveal Exocomets Around 74 Nearby Stars", "url": "https://skyandtelescope.org/astronomy-news/new-images-reveal-exocomets-around-74-nearby-stars/"}, {"title": "MillenniumDB: A graph database engine using domain graphs", "url": "https://github.com/MillenniumDB/MillenniumDB"}, {"title": "Building a Mesh Using Spherical Embedding", "url": "https://andrews.wiki/spherical-mesh"}, {"title": "Hydro: Distributed Programming Framework for Rust", "url": "https://hydro.run/docs/hydro/"}]


In [81]:
print(output)

The top 5 Hacker News stories right now are:

1. **Taking a $15 Casio F91W 5km underwater**: A blog post about testing the water resistance of a cheap Casio watch, pushing it to its limits 5km underwater. (https://www.watchesofespionage.com/blogs/woe-dispatch/casio-f91w-diving-underwater-pressure-test)

2. **Images Reveal Exocomets Around 74 Nearby Stars**: An article from Sky & Telescope about new images that have revealed exocomets orbiting around 74 nearby stars, shedding more light on our understanding of the universe. (https://skyandtelescope.org/astronomy-news/new-images-reveal-exocomets-around-74-nearby-stars/)

3. **MillenniumDB: A graph database engine using domain graphs**: An open-source graph database engine on GitHub that utilizes domain graphs for efficient data storage and querying. (https://github.com/MillenniumDB/MillenniumDB)

4. **Building a Mesh Using Spherical Embedding**: A wiki article about creating a mesh using spherical embedding, a technique for generating 3D